# Subsetting on Arbitrary Dimensions

### Environment Setup
Install the required packages in your virtual environment and import them

In [ ]:
import sys
!{sys.executable} -m pip install netCDF4 harmony-py

In [35]:
from harmony import BBox, Client, Collection, Request, Dimension
from harmony.config import Environment
import os
import netCDF4 as nc4

### Request Helper Functions

In [36]:
# Constructs the Harmony request, subsetting on "lev"
def get_request(minLon, minLat, maxLon, maxLat):
    collection = Collection(id='C1215726323-GES_DISC')
    request = Request(
        collection = collection,
        spatial    = BBox(minLon, minLat, maxLon, maxLat),
        dimensions = [Dimension(name='lev', min=800, max=900)],
        max_results = 1
    )
    return request

# Submit Harmony request and return the result file name
def submit_request(request):
    job_id = harmony_client.submit(request)
    # show the progress bar for the job, get the result URL
    harmony_client.wait_for_processing(job_id, show_progress=True)
    harmony_client.result_urls(job_id, show_progress=True)
    # create a local subdir, download the results 
    dir_name = './harmony_dimension_subset_results'
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    results = harmony_client.download_all(job_id, directory=dir_name, overwrite=True)
    # stash the file names for easy reference
    file_names = [f.result() for f in results]
    # return the first file name
    return file_names[0]

### Request Submission with harmony-py

In [37]:
harmony_client = Client(env=Environment.UAT) # assumes .netrc usage
# Tropics Bounding box
minLon = -160
maxLon = -75
minLat = -10
maxLat = 10
request = get_request(minLon, minLat, maxLon, maxLat)
harmony_client._get_prepared_request(request).url

'https://harmony.uat.earthdata.nasa.gov/C1215726323-GES_DISC/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?forceAsync=true&subset=lat%28-10%3A10%29&subset=lon%28-160%3A-75%29&subset=lev%28800%3A900%29&maxResults=1'

In [38]:
if request.is_valid():
    harmony_fname = submit_request(request)
else:
    harmony_fname = 'Invalid_Request'

 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   0% ] |                                                   | [-]
 [ Processing:   0% ] |                                                   | [\]
 [ Processing:   0% ] |                                                   | [|]
 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   0% ] |                                                   | [-]
 [ Processing:   0% ] |                                                   | [\]
 [ Processing:   0% ] |                                                   | [|]
 [ Processing:   0% ] |                                                   | [/]
 [ Processing:   0% ] |                                                   | [-]
 [ Processing:   0% ] |                                                   | [\]
 [ Processing:   0% ] |                                                   | [|]
 [ Processing:   0% ] |                 

### Request Results
Inspect the results to see how the data has been subsetted

In [39]:
nc4_file=nc4.Dataset(harmony_fname)
print(nc4_file.variables['lev'][:])
print(min(nc4_file.variables['lev']))
print(max(nc4_file.variables['lev']))

[900. 875. 850. 825. 800.]
800.0
900.0
